### testing the functions in the get_othophoto folder

In [1]:
import json
import os
import shutil
import sys
sys.path.append('../get_orthophoto')

from project_metadata import get_all_projects
from start_export import start_export, save_export_job
from status_export import status_export, save_download_url
from download_project import download_project

#### list of projects

In [2]:
projects = get_all_projects()

# trying with selection, but most names are exactly the same on the website
selection = [p if "strinda" in p.lower() else None for p in projects]
selection = [p for p in selection if p is not None]
other_selection = [p if "trondheim 2019" in p.lower() else None for p in projects]
other_selection = [p for p in other_selection if p is not None]
selected = selection + other_selection
print(selected)

['Strinda 1964', 'Trondheim Strinda 1937', 'Trondheim 2019']


### start export

In [3]:
# write down export details
export = {
    "project": 'Trondheim Strinda 1937',
    "resolution" : 1,
    "compression_method": 5,
    "compression_value": 50,
    "mosaic": False,
    }
if False: # request export and save jobID
    JobID = start_export(**export)
    save_export_job(JobID,**export)

#### check status and save download link

In [4]:
# check for current exports:
jobs = os.listdir(os.path.dirname(os.path.dirname(os.getcwd())) + "/data/temp/jobids/")
# exclude folders:
jobs = [j for j in jobs if "." in j]
print(f'available jobs: {jobs}.')
if len(jobs) > 1:
    # by default, take the first job
    current_job = jobs[0]
    # read in json:
    job_path = os.path.dirname(os.path.dirname(os.getcwd())) + "/data/temp/jobids/"
    with open(job_path+current_job, "r") as f:
        job_details = json.load(f)
    complete, url = status_export(job_details["JobID"])
    print(f'The job is done: {complete}, the url is {url}.')

    if complete:
        print("Export complete, JobID moved to archive")
        job_details.pop("JobID")
        save_download_url(url, **job_details)
        # move the job to archive
        shutil.move(job_path+current_job, job_path+"used_jobids/"+current_job)

available jobs: [].


#### save the files

In [6]:
possible_downloads = os.listdir(os.path.dirname(os.path.dirname(os.getcwd())) + "/data/temp/urls/")
# exclude folders
possible_downloads = [p for p in possible_downloads if "." in p]
print(f'current urls: {possible_downloads}.')
if len(possible_downloads) > 0:
    current_download = possible_downloads[0]
    # read in json:
    job_path = os.path.dirname(os.path.dirname(os.getcwd())) + "/data/temp/urls/"
    with open(job_path+current_download, "r") as f:
        job_details = json.load(f)
    job_details

    # download the project
    download_project(**job_details)
    # move the job to archive
    shutil.move(job_path+current_download, job_path+"used_urls/"+current_download)

current urls: ['Download_trondheim strinda 1937_20240306-103723.json'].
